### Importing packages

In [19]:
import numpy as np
import pandas as pd
import torch
import pickle
import os

### Check the data statistics

In [20]:
!ls code/data/parsed

epinion-parsed  melon-parsed  ml-20m-parsed  sketchfab-parsed


In [21]:
with open("code/data/parsed/epinion-parsed", 'rb') as f:
    tr, val, te = pickle.load(f)

In [22]:
n_users, n_items = tr.shape

In [23]:
t = (tr+val+te).tocsr()

In [24]:
n_users, n_items, t.nnz

(21396, 59377, 631422)

In [25]:
t.nnz / (n_users*n_items)

0.000497014282464858

In [26]:
t.data[:] = 1

In [27]:
avg_consumed_items_per_user = t.sum(1).A1.ravel().mean()

In [28]:
avg_consumed_users_per_item = t.sum(0).A1.ravel().mean()

In [29]:
avg_consumed_items_per_user, avg_consumed_users_per_item

(29.51121704991587, 10.634117587618102)

In [30]:
std_items_per_user = t.sum(1).A1.ravel().std()
std_users_per_item = t.sum(0).A1.ravel().std()

In [31]:
std_items_per_user, std_users_per_item

(55.87797105714283, 25.779544411399108)

In [32]:
item_clickcnts = -np.sort(-t.sum(0).A1.ravel())

In [33]:
tot_cnt = item_clickcnts.sum()
i = 0
tc = 0 
for i in range(n_items // 20):
    tc += item_clickcnts[i]
print("%0.2f" % (100 *(tc / tot_cnt)))

38.81


### Check the result of model training

In [59]:
!ls code/best_res/epinion-parsed

bpr  wmf


In [49]:
x = torch.load("code/best_res/epinion-parsed/bpr")

In [50]:
x.keys()

dict_keys(['model', 'param'])

In [51]:
x['param']['test_at_5'][0]

{'precision': 0.014578157607075102,
 'recall': 0.011014155875379619,
 'map': 0.0081052701315048,
 'ndcg': 0.017247632231643804}

In [52]:
x['param']['test_at_10'][0]

{'precision': 0.012481367385471157,
 'recall': 0.018727182060700693,
 'map': 0.005074216956980525,
 'ndcg': 0.018924448806283745}

In [53]:
x['param']['test_at_20'][0]

{'precision': 0.01028023452250851,
 'recall': 0.03156394933751106,
 'map': 0.0030187497246419053,
 'ndcg': 0.02278899024678887}

In [54]:
x['param']['test_at_30'][0]

{'precision': 0.00902149789658456,
 'recall': 0.04196462277279025,
 'map': 0.002188031594659484,
 'ndcg': 0.025940196324878817}

In [55]:
x['param']['test_at_50'][0]

{'precision': 0.0075842194176688514,
 'recall': 0.05889008300837755,
 'map': 0.0014516919472464954,
 'ndcg': 0.03086291755002181}

In [56]:
for kk in [5, 10, 20, 30, 50]:
    for k, v in x['param']['test_at_%d' % kk][0].items():
        print(v)

0.014578157607075102
0.011014155875379619
0.0081052701315048
0.017247632231643804
0.012481367385471157
0.018727182060700693
0.005074216956980525
0.018924448806283745
0.01028023452250851
0.03156394933751106
0.0030187497246419053
0.02278899024678887
0.00902149789658456
0.04196462277279025
0.002188031594659484
0.025940196324878817
0.0075842194176688514
0.05889008300837755
0.0014516919472464954
0.03086291755002181
